In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 100,
    filter_low_freq = 2
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9631.61it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5597.16it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3220926.79it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10322.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5702.03it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 637858.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7459.91it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x107556d30>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x107580710>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.DRMMTKS.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.DRMMTKS()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['top_k'] = 10
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMMTKS(
  (embedding): Embedding(16675, 100)
  (attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=128, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  1702163


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.790]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5187 - normalized_discounted_cumulative_gain@5(0.0): 0.5819 - mean_average_precision(0.0): 0.5332



[Iter-204 Loss-0.490]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5987 - normalized_discounted_cumulative_gain@5(0.0): 0.6472 - mean_average_precision(0.0): 0.61



[Iter-306 Loss-0.382]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6006 - normalized_discounted_cumulative_gain@5(0.0): 0.6582 - mean_average_precision(0.0): 0.6125



[Iter-408 Loss-0.259]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.607 - normalized_discounted_cumulative_gain@5(0.0): 0.6632 - mean_average_precision(0.0): 0.6084



[Iter-510 Loss-0.148]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6156 - normalized_discounted_cumulative_gain@5(0.0): 0.6649 - mean_average_precision(0.0): 0.6207



[Iter-612 Loss-0.105]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6154 - normalized_discounted_cumulative_gain@5(0.0): 0.6655 - mean_average_precision(0.0): 0.6206



[Iter-714 Loss-0.088]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6083 - normalized_discounted_cumulative_gain@5(0.0): 0.6641 - mean_average_precision(0.0): 0.6185



[Iter-816 Loss-0.059]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5778 - normalized_discounted_cumulative_gain@5(0.0): 0.6493 - mean_average_precision(0.0): 0.5995



[Iter-918 Loss-0.040]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6121 - normalized_discounted_cumulative_gain@5(0.0): 0.6726 - mean_average_precision(0.0): 0.6226



[Iter-1020 Loss-0.049]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.609 - normalized_discounted_cumulative_gain@5(0.0): 0.6678 - mean_average_precision(0.0): 0.6164

Cost time: 1233.8079171180725s
